In [40]:
import pandas as pd 
import numpy as np
import os

import pickle

from scipy.sparse import hstack
import faiss

In [41]:
# Indices
with open('data/metadata_indices.pkl', 'rb') as f:
    metadata_indices = pickle.load(f)

with open('data/overview_indices.pkl', 'rb') as f:
    overview_indices = pickle.load(f)

# Metadata Vector Matrices
with open('data/metadata_vector_matrix_all.pkl', 'rb') as f:
    metadata_matrix_all = pickle.load(f)

with open('data/metadata_vector_matrix_people.pkl', 'rb') as f:
    metadata_matrix_people = pickle.load(f)

with open('data/metadata_vector_matrix_theme.pkl', 'rb') as f:
    metadata_matrix_theme = pickle.load(f)

# Plot Embeddings
with open('data/overview_embedding_matrix.pkl', 'rb') as f:
    overview_matrix = pickle.load(f)

0. Check Indices if matching

In [42]:
# Compare the two objects using the Pandas equals() method
if metadata_indices.equals(overview_indices):
    print("The contents of metadata_indices.pkl and overview_indices.pkl are the same.")
    indices = overview_indices
else:
    print("The contents of metadata_indices.pkl and overview_indices.pkl are different.")

The contents of metadata_indices.pkl and overview_indices.pkl are the same.


In [43]:
output_path = os.path.join('data', 'final', 'indices.pkl')
with open(output_path, 'wb') as f:
    pickle.dump(indices, f)

1. Build Matrix and Faiss Index for Plot + All Metadata

In [44]:
metadata_matrix = metadata_matrix_all.copy()

combined_matrix_sparse = hstack([np.array(overview_matrix).astype(np.float32), metadata_matrix.astype(np.float32)])
combined_matrix_dense = combined_matrix_sparse.toarray()

faiss.normalize_L2(combined_matrix_dense)
embedding_dimension = combined_matrix_dense.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dimension)
faiss_index.add(combined_matrix_dense)

In [45]:
output_path = os.path.join('data', 'final', 'all_matrix_dense.pkl')
with open(output_path, 'wb') as f:
    pickle.dump(combined_matrix_dense, f)

In [46]:
output_path = os.path.join('data', 'final', 'all_faiss.bin')
faiss.write_index(faiss_index, output_path)

2. Build Matrix and Faiss Index for Plot + Themes (Genres, Keywords)

In [47]:
metadata_matrix = metadata_matrix_theme.copy()

combined_matrix_sparse = hstack([np.array(overview_matrix).astype(np.float32), metadata_matrix.astype(np.float32)])
combined_matrix_dense = combined_matrix_sparse.toarray()

faiss.normalize_L2(combined_matrix_dense)
embedding_dimension = combined_matrix_dense.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dimension)
faiss_index.add(combined_matrix_dense)

In [48]:
output_path = os.path.join('data', 'final', 'theme_plot_matrix_dense.pkl')
with open(output_path, 'wb') as f:
    pickle.dump(combined_matrix_dense, f)

In [49]:
output_path = os.path.join('data', 'final', 'theme_plot_faiss.bin')
faiss.write_index(faiss_index, output_path)

3. Build Matrix and Faiss Index for Cast + Crew

In [50]:
metadata_matrix_dense_people = metadata_matrix_people.astype(np.float32).toarray()

faiss.normalize_L2(metadata_matrix_dense_people) 
embedding_dimension_people = metadata_matrix_dense_people.shape[1]
faiss_index_people = faiss.IndexFlatIP(embedding_dimension_people)
faiss_index_people.add(metadata_matrix_dense_people)

In [51]:
output_path = os.path.join('data', 'final', 'people_matrix_dense.pkl')
with open(output_path, 'wb') as f:
    pickle.dump(metadata_matrix_dense_people, f)

In [52]:
output_path = os.path.join('data', 'final', 'people_faiss.bin')
faiss.write_index(faiss_index_people, output_path)